<a href="https://colab.research.google.com/github/DarshanSajeev/COMP2611/blob/main/Copy_of_SearchExercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   327k      0 --:--:-- --:--:-- --:--:--  321k


In [20]:
class Robot:
    def __init__(self, location, carried_items=None):
        if carried_items is None:
            carried_items = []  # Default to an empty list if no items are specified
        self.location = location
        self.carried_items = carried_items

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, self.carried_items ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )
    
class State:
    def __init__(self, robot, doors, room_contents):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents  # Ensure this attribute is passed and initialized

    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )

In [ ]:
ROOM_CONTENTS = {
    'Storage' : {'M3','P2','E2', 'M6', 'P4'},
    'Office' :{'keycard'},
    'English' : {'E1', 'E3', 'E4'},
    'Maths' : {'M1', 'M2', 'M4', 'M5'},
    'Physics' : {'P1', 'P3'},
}


# Define the rooms that need to be unlocked and locked
storage_rooms = ['English', 'Maths', 'Physics', 'Office']

# Initialize the doors list with the locked rooms (History and Physics)
# Initialize the doors list with the locked rooms (History and Physics)
DOORS = [
    # History and Physics are locked and need the keycard to access
    Door('Storage', 'Physics', doorkey='keycard', locked=True),
    Door('Storage', 'Office'),
    Door('Storage', 'Maths'),
    Door('Storage', 'English'),
]



In [22]:
from copy import deepcopy

class RobotWorker(SearchProblem):

    def __init__(self, state, goal_item_locations):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions(self, state):
        robot_location = state.robot.location
        actions = []

        # Can put down any carried item if it's the correct room
        for item in state.robot.carried_items:
            # Check if the item belongs to the current room
            if item in self.goal_item_locations.get(robot_location, set()):
                actions.append(("put down", item))

        # Can pick up any item in the current room
        for item in state.room_contents[robot_location]:
            # If the item needs to be sorted, it can be picked up
            if item not in state.robot.carried_items:
                actions.append(("pick up", item))

        # If there is an unlocked door between robot location and another location, robot can move
        for door in state.doors:
            if not door.locked and robot_location in door.goes_between:
                actions.append(("move to", door.other_loc[robot_location]))

        # Return the list of possible actions
        return actions

    def successor(self, state, action):
        next_state = deepcopy(state)
        act, target = action

        if act == "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)
        
        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        for room, contents in self.goal_item_locations.items():
            for item in contents:
                if item not in state.room_contents[room]:
                    return False
        return True

    def display_state(self, state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)


In [23]:
# Initial State Setup
initial_robot = Robot('Storage')
initial_state = State(initial_robot, DOORS, ROOM_CONTENTS)

# Goal Locations Setup
goal_item_locations = {
    'Storage' : {'M1','P2','E3'},
}

# Instantiate the RobotWorker with the initial state and goal
RW_PROBLEM = RobotWorker(initial_state, goal_item_locations)

poss_acts = RW_PROBLEM.possible_actions( RW_PROBLEM.initial_state )
poss_acts

for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM.successor( RW_PROBLEM.initial_state, act )
    RW_PROBLEM.display_state(next_state)
    print()

Action ('pick up', 'E2') leads to the following state:
Robot location: Storage
Robot carrying: ['E2']
Room contents: {'Storage': {'P2', 'M3'}, 'Office': {'keycard'}, 'English': {'E3', 'E1'}, 'Maths': {'M1', 'M2'}, 'Physics': {'P1'}}

Action ('pick up', 'M3') leads to the following state:
Robot location: Storage
Robot carrying: ['M3']
Room contents: {'Storage': {'P2', 'E2'}, 'Office': {'keycard'}, 'English': {'E3', 'E1'}, 'Maths': {'M1', 'M2'}, 'Physics': {'P1'}}

Action ('pick up', 'P2') leads to the following state:
Robot location: Storage
Robot carrying: ['P2']
Room contents: {'Storage': {'M3', 'E2'}, 'Office': {'keycard'}, 'English': {'E3', 'E1'}, 'Maths': {'M1', 'M2'}, 'Physics': {'P1'}}

Action ('move to', 'Office') leads to the following state:
Robot location: Office
Robot carrying: []
Room contents: {'Storage': {'P2', 'M3', 'E2'}, 'Office': {'keycard'}, 'English': {'E3', 'E1'}, 'Maths': {'M1', 'M2'}, 'Physics': {'P1'}}

Action ('move to', 'Maths') leads to the following state:
R

In [26]:
search( RW_PROBLEM, 'BF/FIFO', 100000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
....
:-)) *SUCCESS* ((-:

Path length = 8
Goal state is:
Robot location: Storage
Robot carrying: []
Room contents: {'Storage': {'M1', 'E3', 'M3', 'P2', 'E2'}, 'Office': {'keycard'}, 'English': {'E1'}, 'Maths': {'M2'}, 'Physics': {'P1'}}
The action path to the solution is:
    ('move to', 'Maths')
    ('pick up', 'M1')
    ('move to', 'Storage')
    ('put down', 'M1')
    ('move to', 'English')
    ('pick up', 'E3')
    ('move to', 'Storage')
    ('put down', 'E3')


SEARCH SPACE STATS:
Total nodes generated          =    14408  (includes start)
Nodes discarded by loop_check  =     4494  (9914 distinct states added to queue)
Nodes tested (by goal_test)    =     4833  (4832 expanded + 

{'args': {'problem': 'RobotWorker',
  'mode': 'BF/FIFO',
  'max_nodes': 100000,
  'loop_check': True,
  'randomise': False,
  'cost': None,
  'heuristic': None,
  'dots': True},
 'result': {'termination_condition': 'GOAL_STATE_FOUND',
  'goal_state': ("('Storage', [])", ["('door', {'Storage', 'Physics'}, 'keycard', True)", "('door', {'Storage', 'Office'}, None, False)", "('door', {'Storage', 'Maths'}, None, False)", "('door', {'Storage', 'English'}, None, False)"], {'Storage': {'M1', 'E3', 'M3', 'P2', 'E2'}, 'Office': {'keycard'}, 'English': {'E1'}, 'Maths': {'M2'}, 'Physics': {'P1'}}),
  'path': [('move to', 'Maths'),
   ('pick up', 'M1'),
   ('move to', 'Storage'),
   ('put down', 'M1'),
   ('move to', 'English'),
   ('pick up', 'E3'),
   ('move to', 'Storage'),
   ('put down', 'E3')],
  'path_length': 8},
 'search_stats': {'nodes_generated': 14408,
  'nodes_tested': 4833,
  'nodes_discarded': 4494,
  'distinct_states_seen': 9914,
  'nodes_left_in_queue': 5081,
  'time_taken': 2.2760

In [25]:
# Print the final output of all books in all rooms
for room, books in ROOM_CONTENTS.items():
    print(f"Books in {room}: {', '.join(books)}")

Books in Storage: E2, M3, P2
Books in Office: keycard
Books in English: E3, E1
Books in Maths: M1, M2
Books in Physics: P1


In [30]:
def cost(path, state):
        return len(path)

# cost_robot = search( RW_PROBLEM, 'BF/FIFO', 30000, cost=cost, loop_check=True, return_info=True)

def manhattan_heuristic(state):
    total_distance = 0

    for room, items in state.room_contents.items():
        for item in items:
            if item in goal_item_locations:  # Check if item has a designated goal room
                goal_room = goal_item_locations[item]
                if room != goal_room:  # If item is not in its goal room, calculate distance
                    total_distance += abs(state.room_positions[room][0] - state.room_positions[goal_room][0]) + \
                                      abs(state.room_positions[room][1] - state.room_positions[goal_room][1])
    
    return total_distance


manhattan_robot = search( RW_PROBLEM, 'DF/LIFO', 100000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=manhattan_heuristic)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=cost, heuristic=manhattan_heuristic
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...
:-)) *SUCCESS* ((-:

Path length = 8
Goal state is:
Robot location: Storage
Robot carrying: []
Room contents: {'Storage': {'M1', 'E3', 'M3', 'P2', 'E2'}, 'Office': {'keycard'}, 'English': {'E1'}, 'Maths': {'M2'}, 'Physics': {'P1'}}
Cost of reaching goal: 8
The action path to the solution is:
    ('move to', 'English')
    ('pick up', 'E3')
    ('move to', 'Storage')
    ('move to', 'Maths')
    ('pick up', 'M1')
    ('move to', 'Storage')
    ('put down', 'E3')
    ('put down', 'M1')


SEARCH SPACE STATS:
Total nodes generated          =     9774  (includes start)
Nodes discarded by loop_check  =     3023  (6751 distinct states added to queue)
Nodes tested (by goal